In [5]:
# 필요한 패키지(라이브러리)를 구글 드라이버에 설치하였는가?
IS_INSTALL_ENV = True

In [6]:
import os, sys  
from google.colab import drive  
drive.mount('/content/drive')   

# colab의 os 폴더 위치
pg_path = '/content/dataset' 

# if IS_NOT_INSTALL_ENV == True:
# Google Drive에서 설치된 파일 위치 - 미리 해당 폴더를 생성해야 한다.
os.symlink('/content/drive/My Drive/Colab Notebooks/Project/AI 속기사/dataset/', pg_path)  
sys.path.insert(0, pg_path)  


# /content/drive/My Drive/Colab Notebooks/Project/AI 속기사/dataset/KlecSpeech/

Mounted at /content/drive


In [7]:
# colab의 os 폴더 위치
pg_path = '/content/packages' 

# if IS_NOT_INSTALL_ENV == True:
# Google Drive에서 설치된 파일 위치 - 미리 해당 폴더를 생성해야 한다.
os.symlink('/content/drive/My Drive/Colab Notebooks/packages', pg_path)  
sys.path.insert(0, pg_path)  

In [8]:
if IS_INSTALL_ENV == False:
# !pip install transformers 대신 영구 설치
# os.symlink로 걸어놓은 위치에서 pg_path로 연결된 폴더에 영구 설치한다. 1번만 설치하면 2번 설치할 필요 없음.
    !pip install --target=$pg_path datasets
    !pip install --target=$pg_path transformers  
    !pip install --target=$pg_pat jiwer

## Create Wav2Vec2CTCTokenizer

In [9]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import tqdm
import glob

In [10]:
dir_path = ['./dataset/KlecSpeech/csv/', './dataset/csv/']
file_prefix = ['klecspeech_ko_500', 'order_speech_ko1000']

In [23]:
# 모든 csv 파일 리스트를 가져온다.
def get_all_filelist(dir_path):
    csv_files = []
    for i in range(len(dir_path)):
        csv_list = os.listdir(dir_path[i])
        for file in csv_list:
            csv_files.append(dir_path[i]+file)
    return csv_files



In [24]:
# 결과를 확인한다.
all_files = get_all_filelist(dir_path)

In [40]:
# get_filelist 
# start_idx부터 step_num 건수 만큼 파일리스트 추가
def get_filelist(csv_list, start_idx, step_num):

    new_list = []
    if len(csv_list) == 0:
        print("array size is 0")
        return None
    else:
        for i in range(start_idx, start_idx + step_num):
            new_list.append(csv_list[i])
        return new_list

In [27]:
def load_datasets(files):
    # files가 list로 되어 있으면 한번에 합쳐서 load해온다.
    dataset = load_dataset('csv',data_files=files, split='train', sep='\t')
    return dataset

In [29]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [30]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [73]:
import json

clip = 2

for i in range(0, len(all_files), clip):
    filelist = get_filelist(all_files, i, clip)
    print(filelist)
    data_csv = load_datasets(filelist)
    print(data_csv)
    remove_spectial_char_data = data_csv.map(remove_special_characters )
    df = pd.DataFrame(remove_spectial_char_data)

    char_vocab = remove_spectial_char_data.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=remove_spectial_char_data.column_names,
    )


    vocab_list = list(set(char_vocab["vocab"][0]))
    vocab_dict = {v: k for k, v in enumerate(vocab_list)}
    vocab_dict["|"] = vocab_dict[" "]
    del vocab_dict[" "]
    vocab_dict["[UNK]"] = len(vocab_dict)
    vocab_dict["[PAD]"] = len(vocab_dict)

    print(vocab_dict)
    with open(f'vocab_{i:03d}.json_temp', 'w') as vocab_file:
        json.dump(vocab_dict, vocab_file)


['./dataset/KlecSpeech/csv/klecspeech_ko_500_000.csv', './dataset/KlecSpeech/csv/klecspeech_ko_500_001.csv']


Using custom data configuration default-6202ba9b52256026
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-6202ba9b52256026/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


Dataset({
    features: ['filename', 'text', 'array'],
    num_rows: 20
})


  0%|          | 0/20 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

...

Using custom data configuration default-ffe45d03b870e6b3
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-ffe45d03b870e6b3/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


Dataset({
    features: ['filename', 'text', 'array'],
    num_rows: 9
})


  0%|          | 0/9 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

{'줘': 0, '리': 1, '꺼': 2, '음': 3, '오': 4, '는': 5, '당': 6, '침': 7, '씨': 8, '까': 9, '남': 10, '켜': 11, '있': 12, '강': 13, '대': 14, '좀': 15, '이': 16, '크': 17, '브': 18, '핑': 19, '주': 20, '말': 21, '영': 22, '장': 23, '시': 24, '궁': 25, '번': 26, '보': 27, '나': 28, '해': 29, '지': 30, '늘': 31, '정': 32, '체': 33, '구': 34, '수': 35, '알': 36, '날': 38, '등': 39, '고': 40, '로': 41, '균': 42, '신': 43, '점': 44, '상': 45, '때': 46, '린': 47, '아': 48, '다': 49, '흐': 50, '봐': 51, '금': 52, '졌': 53, '어': 54, '취': 55, '량': 56, '평': 57, '가': 58, '|': 37, '[UNK]': 59, '[PAD]': 60}


In [80]:
json_files = glob.glob('*.json_temp')
print(json_files)

vocab_all = {}
for filename in json_files:
    with open(filename, 'r') as vocab_file:
        vocab_dict = json.load(vocab_file)
        print(type(vocab_dict))
        vocab_all.update(vocab_dict)

# 번호 재배열
vocab_all = {v: k for k, v in enumerate(vocab_all)}
print(len(vocab_all))

print(vocab_all)

with open(f'vocab.json', 'w') as vocab_file:  
    json.dump(vocab_all, vocab_file)


['vocab_002.json_temp', 'vocab_000.json_temp']
<class 'dict'>
<class 'dict'>
195
{'줘': 0, '리': 1, '꺼': 2, '음': 3, '오': 4, '는': 5, '당': 6, '침': 7, '씨': 8, '까': 9, '남': 10, '켜': 11, '있': 12, '강': 13, '대': 14, '좀': 15, '이': 16, '크': 17, '브': 18, '핑': 19, '주': 20, '말': 21, '영': 22, '장': 23, '시': 24, '궁': 25, '번': 26, '보': 27, '나': 28, '해': 29, '지': 30, '늘': 31, '정': 32, '체': 33, '구': 34, '수': 35, '알': 36, '날': 37, '등': 38, '고': 39, '로': 40, '균': 41, '신': 42, '점': 43, '상': 44, '때': 45, '린': 46, '아': 47, '다': 48, '흐': 49, '봐': 50, '금': 51, '졌': 52, '어': 53, '취': 54, '량': 55, '평': 56, '가': 57, '|': 58, '[UNK]': 59, '[PAD]': 60, '떤': 61, '비': 62, '새': 63, '세': 64, '더': 65, '잘': 66, '와': 67, '쁜': 68, '았': 69, '검': 70, '에': 71, '으': 72, '뛰': 73, '되': 74, '을': 75, '면': 76, '도': 77, '분': 78, '열': 79, '력': 80, '힘': 81, '뻐': 82, '치': 83, '것': 84, '노': 85, '던': 86, '막': 87, '저': 88, '줄': 89, '색': 90, '러': 91, '마': 92, '너': 93, '근': 94, '함': 95, '각': 96, '쑥': 97, '내': 98, '쏙': 99, '속': 100, '절': 101, 

In [21]:
# print(data_csv)
# for i in range(len(data_csv)):
#     print(data_csv['filename'][i], data_csv['text'][i])

Dataset({
    features: ['filename', 'text', 'array'],
    num_rows: 29
})
./dataset/KlecSpeech/Validation/D01/E01/S000028/000000.wav 여러분 안녕하세요
./dataset/KlecSpeech/Validation/D01/E01/S000028/000001.wav 친구들이 독해의 열매를 맺어 독해력이 쑥쑥 자랄 수 있도록 도와주는 혜은 선생님 이에요 오늘은 어떤 글을 읽을까요 생각의 힘을 키워줄 생각 씨앗입니다
./dataset/KlecSpeech/Validation/D01/E01/S000028/000002.wav 오늘의 생각 씨앗은요 히읗 비읍 이에요 선생님은 어렸을 때 이 생각 씨앗을 입고 싶었는데 어머니께서 사주시지 않았어요 금방 큰다면서 나중에 더 크면 사주겠다던 어머니의 말씀이 기억나는데요 오늘의 이야기 함께 읽어봐요
./dataset/KlecSpeech/Validation/D01/E01/S000028/000003.wav 색동저고리 함께 읽어보도록 하겠습니다
./dataset/KlecSpeech/Validation/D01/E01/S000028/000004.wav 와 엄마 너무 예뻐요
./dataset/KlecSpeech/Validation/D01/E01/S000028/000005.wav 엄마께서 보여주신 새 한복은 내 마음에 쏙 들었어요
./dataset/KlecSpeech/Validation/D01/E01/S000028/000006.wav 지난 명절에 작아진 한복을 입지 못해 속상했더니 엄마께서 새 한복을 사주신 거예요
./dataset/KlecSpeech/Validation/D01/E01/S000028/000007.wav 새로 산 한복은 빨강 치마에 색동저고리인데 저고리 바탕은 노란색이고 팔 부분은 파란색 노란색 빨간색 하얀색 검은색의 다섯 가지 색으로 되어 있어요
./dataset/KlecSpeech/Validation/D01/E01/S000028/

In [11]:
df = pd.DataFrame(remove_spectial_char_data)
df.head()

,filename,text,array
0,./dataset/KlecSpeech/Validation/D01/E01/S00002...,여러분 안녕하세요,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,./dataset/KlecSpeech/Validation/D01/E01/S00002...,친구들이 독해의 열매를 맺어 독해력이 쑥쑥 자랄 수 있도록 도와주는 혜은 선생님 이...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,./dataset/KlecSpeech/Validation/D01/E01/S00002...,오늘의 생각 씨앗은요 히읗 비읍 이에요 선생님은 어렸을 때 이 생각 씨앗을 입고 싶...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,./dataset/KlecSpeech/Validation/D01/E01/S00002...,색동저고리 함께 읽어보도록 하겠습니다,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,./dataset/KlecSpeech/Validation/D01/E01/S00002...,와 엄마 너무 예뻐요,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [19]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json",
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

## Create XLSR-Wav2Vec2 Feature Extractor

In [20]:
import torch

In [21]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)

In [22]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [32]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [33]:
# processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-ready")

## add audio array

In [23]:
import librosa
def load_audio(batch):
    batch['array'], _ = librosa.load(batch['filename'],sr=16000)
    return batch

In [24]:
# array=remove_spectial_char_data['array'][0]
# type(array)
audio_data = remove_spectial_char_data.map(load_audio)

  0%|          | 0/500 [00:00<?, ?ex/s]

In [25]:
# Dataset type이 맞는지 확인할 것
print(type(audio_data))
print(type(remove_spectial_char_data))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


## Preprocess Data

In [26]:
def prepare_dataset(batch):

    # batched output is "un-batched"
    batch["input_values"] = processor(batch["array"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [28]:
order_voice = audio_data.map(
    prepare_dataset,
    remove_columns=remove_spectial_char_data.column_names,
    num_proc=4 # jupyter notebook에서는 에러발생 #colab 또는 하드웨어 준비된 환경에서 사용 가능한 옵션, 병렬로 함수를 실행할 수있다. 
)

#0:   0%|          | 0/125 [00:00<?, ?ex/s]

#2:   0%|          | 0/125 [00:00<?, ?ex/s]

#1:   0%|          | 0/125 [00:00<?, ?ex/s]

#3:   0%|          | 0/125 [00:00<?, ?ex/s]

In [29]:
order_voice

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 500
})

# Training

## Set-up Trainer

In [30]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.cuda.HalfTensor]]]) -> Dict[str, torch.cuda.HalfTensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [31]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [34]:
from datasets import load_dataset, load_metric, Audio

wer_metric = load_metric("wer")

In [35]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Import Model

In [36]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'project_hid.weight', 'project_hid.bias', 'quantizer.codevectors', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

XLSR-Wav2Vec2의 첫 번째 구성 요소는 원시 음성 신호에서 음향적으로 의미가 있지만 문맥적으로 독립적인 기능을 추출하는 데 사용되는 CNN 계층 스택으로 구성됩니다.  
모델의 이 부분은 사전 교육 중에 이미 충분히 훈련되었으며 논문에 명시된 바와 같이 더 이상 미세 조정할 필요가 없습니다. 따라서 특징 추출 부분의 모든 파라미터에 대해 require_grad를 False로 설정할 수 있다.

In [37]:
model.freeze_feature_extractor()

/content/packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


메모리를 절약하기 위해 그라데이션 체크포인팅을 활성화

In [38]:
model.gradient_checkpointing_enable()

## TrainingArguments

In [47]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-ko-demo",
  output_dir="./wav2vec2-large-xlsr-ko-demo",
  group_by_length=True,
  per_device_train_batch_size=8, # 원래는 16인데 cuda memory가 15G를 요구하기 때문에 절반으로 줄였다.
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=5,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [48]:
from transformers import Trainer
from torch.utils.data import random_split

ds_size = len(order_voice)
train_size = int(ds_size*0.8)
val_size = ds_size - train_size
train_ds, val_ds = random_split(order_voice,[train_size,val_size])

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


## Model Training

In [45]:
# True가 나와야 cuda 환경을 사용할 수 있다.
torch.cuda.is_available()

True

In [ ]:
# cuda cache를 초기화 한다.
import gc
torch.cuda.empty_cache()
gc.collect()

In [49]:
# RuntimeError: CUDA out of memory가 발생한다면, per_device_train_batch_size를 줄여본다.
# 단 batch_size를 줄이는 경우, 학습이 늦어지고, 학습률에 영향이 있을 수 있다.
trainer.train()

/content/packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 400
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 125


Step,Training Loss,Validation Loss,Wer
100,21.559300,17.424671,1.000000


***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo/checkpoint-100
Configuration saved in ./wav2vec2-large-xlsr-ko-demo/checkpoint-100/config.json
Model weights saved in ./wav2vec2-large-xlsr-ko-demo/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo/checkpoint-100/preprocessor_config.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=125, training_loss=33.884866333007814, metrics={'train_runtime': 354.8585, 'train_samples_per_second': 5.636, 'train_steps_per_second': 0.352, 'total_flos': 4.0931276704186176e+17, 'train_loss': 33.884866333007814, 'epoch': 5.0})

CTC 손실을 사용하여 더 큰 데이터 세트에서 더 큰 모델을 미세 조정하려면 [여기서](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 공식 음성 인식 예를 살펴봐야 한다.